# Step 1: Importing Essential Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")

# Step 2: Loading data and Making labels

In [2]:
#Dataset used: https://www.kaggle.com/kmader/skin-cancer-mnist-ham10000
image_path = "archive/hmnist_28_28_RGB.csv"
meta_path = "archive/HAM10000_metadata.csv"

In [3]:
df = pd.read_csv(image_path)
metadata = pd.read_csv(meta_path)

In [4]:
df.tail()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
10010,183,165,181,182,165,180,184,166,182,188,...,208,185,187,208,186,186,206,187,189,0
10011,2,3,1,38,33,32,121,104,103,132,...,96,79,76,24,23,21,3,4,1,0
10012,132,118,118,167,149,149,175,156,160,184,...,204,181,178,181,159,153,172,151,145,0
10013,160,124,146,164,131,152,167,127,146,169,...,185,162,167,184,157,166,185,162,172,0
10014,175,142,121,181,150,134,181,150,133,178,...,159,79,82,174,137,125,175,139,126,6


In [5]:
metadata.tail()

,lesion_id,image_id,dx,dx_type,age,sex,localization
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face
10014,HAM_0003521,ISIC_0032258,mel,histo,70.0,female,back


In [6]:
#drop NA values
meta_set = metadata[["age", "sex", "localization"]]
df = pd.concat([meta_set, df], axis = 1)
df = df.dropna()
df.tail()

,age,sex,localization,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
10010,40.0,male,abdomen,183,165,181,182,165,180,184,...,208,185,187,208,186,186,206,187,189,0
10011,40.0,male,abdomen,2,3,1,38,33,32,121,...,96,79,76,24,23,21,3,4,1,0
10012,40.0,male,abdomen,132,118,118,167,149,149,175,...,204,181,178,181,159,153,172,151,145,0
10013,80.0,male,face,160,124,146,164,131,152,167,...,185,162,167,184,157,166,185,162,172,0
10014,70.0,female,back,175,142,121,181,150,134,181,...,159,79,82,174,137,125,175,139,126,6


# Step 3: Train Test Split

In [7]:
# Split into train, validation, and test.
np.random.seed(2070404)

# Shuffle all records.
df_shuffle = df.sample(frac = 1)

# Create split counts.
splits = np.multiply(len(df_shuffle), (0.6,0.2,0.2)).astype(int)
print(f"Split counts (train/ validation/ test): {splits}")

# Create split data sets.
train_set, valid_set, test_set = np.split(df_shuffle, [splits[0], splits[0] + splits[1]])

# Reset split set indicies.
train_set.reset_index(drop = True, inplace = True)
valid_set.reset_index(drop = True, inplace = True)
test_set.reset_index(drop = True, inplace = True)

Split counts (train/ validation/ test): [5974 1991 1991]


In [8]:
#separate features and labels
y_train_clean = train_set['label']
x_train_clean = train_set.drop(columns=['label'])

y_valid = valid_set['label']
x_valid = valid_set.drop(columns=['label'])

y_test = test_set['label']
x_test = test_set.drop(columns=['label'])

columns = list(x_train_clean)

# Step 4: Preprocessing

In [9]:
"""
The following labels are malignant
label 0: AKIEC (Actinic Keratoses and Intraepithelial Carcinoma)
label 1: BCC (Basal Cell Carcinoma)
label 6: MEL (Melanoma)
"""

def classify_label_malignant(df_row):
    """ 
    return whether a record is malignant
    
    Args:
    df_row: single record
    
    
    Returns:
    1.0 if malignant
    0.0 if not malignant
    """
    
    if df_row == 0 or df_row == 1 or df_row == 6:
        return 1.0
    else:
        return 0.0

In [10]:
#apply classify_label_malignant to train, validation, and test set label to gain binary classification of whether a record is malignant


y_train_clean = y_train_clean.apply(classify_label_malignant)
y_valid = y_valid.apply(classify_label_malignant)
y_test = y_test.apply(classify_label_malignant)

y_train_clean.tail()


5969    1.0
5970    0.0
5971    0.0
5972    1.0
5973    0.0
Name: label, dtype: float64

In [11]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#create a new train set that over-sample the minority class(es) by picking samples at random with replacement
oversample = RandomOverSampler()
x_train_oversample, y_train_oversample  = oversample.fit_resample(x_train_clean, y_train_clean)

#create encoder to encode categorical features as one-hot arrays
ohe = OneHotEncoder()
#create scaler to standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()

#apply encoder and scaler on clean (original) dataset's metadata
categorical_data_clean = ohe.fit_transform(x_train_clean[['sex', 'localization']]).toarray()
numerical_data_clean = scaler.fit_transform(x_train_clean[['age']].values)
train_tabular_clean = np.concatenate([categorical_data_clean, numerical_data_clean], axis = 1)
x_train_clean = x_train_clean.iloc[:,3:]

#apply encoder and scaler on oversample dataset's metadata
categorical_data_oversample = ohe.transform(x_train_oversample[['sex', 'localization']]).toarray()
numerical_data_oversample = scaler.transform(x_train_oversample[['age']].values)
train_tabular_oversample = np.concatenate([categorical_data_oversample, numerical_data_oversample], axis = 1)
x_train_oversample = x_train_oversample.iloc[:,3:]

#apply encoder and scaler on validation dataset's metadata
categorical_data_valid = ohe.transform(x_valid[['sex', 'localization']]).toarray()
numerical_data_valid = scaler.transform(x_valid[['age']].values)
valid_tabular_set = np.concatenate([categorical_data_valid, numerical_data_valid], axis = 1)
x_valid = x_valid.iloc[:,3:]


In [12]:
print(train_tabular_clean.shape)
print(x_train_clean.shape)

(5974, 19)
(5974, 2352)


In [13]:
print(train_tabular_oversample.shape)
print(x_train_oversample.shape)

(9636, 19)
(9636, 2352)


In [14]:
print(valid_tabular_set.shape)
print(x_valid.shape)

(1991, 19)
(1991, 2352)


In [15]:
import matplotlib.pyplot as plt
import random

x_train_clean = np.array(x_train_clean, dtype=np.uint8).reshape(-1,28,28,3)
x_train_oversample = np.array(x_train_oversample, dtype=np.uint8).reshape(-1,28,28,3)
x_valid = np.array(x_valid, dtype=np.uint8).reshape(-1,28,28,3)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow import keras
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [17]:
x_valid_augm = x_valid / 255.0

def aug_image(x_train_set, tabular_set, y_train_set, contrast_factor = 2, delta = 0.1, flip_flag = True):
    """ 
    apply transformaions and augmentations
    
    Args:
    x_train_set: dataset to be augmented and transformed
    tabular_set: metadata of the dataset
    y_train_set: labels of the dataset
    contrast_factor: contrast factor to be used
    delta: delta value to be used
    flip_flag: whether to apply flipping transformation to the dataset
    
    
    Returns:
    x_train_set_augm: transformed and augmented dataset
    tabular_set_augm: metadata of the transformed and augmented dataset
    y_train_set_augm: labels of the transformed and augmented dataset
    
    """
    
    
    tf.random.set_seed(1234)

    #transform image to greyscale
    x_train_set_augm = x_train_set / 255.0

    #change delta
    x_train_set_augm = tf.image.adjust_brightness(x_train_set_augm, delta = delta)

    #change contrast
    x_train_set_augm = tf.image.adjust_contrast(x_train_set_augm, contrast_factor = contrast_factor)

    #flip image
    if flip_flag:
        x_train_set_augm = tf.image.random_flip_left_right(x_train_set_augm)

    #concatenate original dataset and augmented dataset
    x_train_set_augm = tf.concat([x_train_set, x_train_set_augm], axis = 0)
    
    #concatenate original dataset metadata with itselt
    tabular_set_augm = tabular_set
    tabular_set_augm = tf.concat([tabular_set_augm, tabular_set], axis = 0)

    #concatenate original dataset labels with itselt
    y_train_set_augm = y_train_set
    y_train_set_augm = tf.concat([y_train_set, y_train_set_augm], axis = 0)


    #shuffle dataset
    shuffle = tf.random.shuffle(tf.range(tf.shape(x_train_set_augm)[0], dtype = tf.int32))
    x_train_set_augm = tf.gather(x_train_set_augm, shuffle)
    tabular_set_augm = tf.gather(tabular_set_augm, shuffle)
    y_train_set_augm = tf.gather(y_train_set_augm, shuffle).numpy() #also transforms y_train to numpy array

    #print(x_train_set_augm.shape)
    
    return x_train_set_augm, tabular_set_augm, y_train_set_augm

# Step 5: Model Building

### 5.1: build a neural network model for metadata features (tabular data)

In [18]:
def build_tabular_model(input_dim = 19, activation_func = "relu", tabular_dense_list = [64]):    
    """ 
    build a neural network model using keras
    
    Args:
    input_dim: dimension of the input
    activation_func: activation function to be used in the hidden dense layers
    tabular_dense_list: a list of dense layer units to be added
    
    Returns:
    model: A tf.keras model
    
    """
    
    model = tf.keras.Sequential()

    for i in range(len(tabular_dense_list)):
        #add dense layer 1
        if i == 0:
            model.add(tf.keras.layers.Dense(
                units = tabular_dense_list[i],
                activation = activation_func,
                input_dim = input_dim))
        #add additional dense layer(s)
        else:
            model.add(tf.keras.layers.Dense(
                units = tabular_dense_list[i],
                activation = activation_func))
            
    return model
    

### 5.2: build a CNN model for images

In [19]:
def build_cnn_model(kernel_size = 2, pool_size = 2):
    """
    Build a CNN model using Keras.

    Args:
    kernel_size: convolution layer kernel size
    pool_size: pooling layer pool size

    Returns:
    model: A tf.keras model
    """
    
    model = tf.keras.Sequential()

    #add convolution layer 1
    model.add(tf.keras.layers.Conv2D(
        filters = 32, kernel_size = (kernel_size, kernel_size),
        strides=(1,1), padding='same',
        data_format = 'channels_last',
        input_shape = (28, 28, 3),  # Updated input shape for RGB
        name='conv_1', activation='relu'))

    #add pooling layer 1
    model.add(tf.keras.layers.MaxPool2D(
        pool_size = (pool_size, pool_size), name = 'pool_1'))

    #add convolution layer 2
    model.add(tf.keras.layers.Conv2D(
        filters = 64, kernel_size = (kernel_size, kernel_size),
        strides = (1,1), padding = 'same',
        name = 'conv_2', activation = 'relu'))

    #add pooling layer 2
    model.add(tf.keras.layers.MaxPool2D(
        pool_size = (pool_size, pool_size), name = 'pool_2'))

    #add flattening layer
    model.add(tf.keras.layers.Flatten())


    #add dense layer 1
    model.add(tf.keras.layers.Dense(
        units = 1024, name = 'fc_1', 
        activation = 'relu'))

    #dropout
    model.add(tf.keras.layers.Dropout(rate = 0.5))
    
    #add dense layer 2
    model.add(tf.keras.layers.Dense(
        units = 1024, name = 'dense2', 
        activation = 'relu'))
                
    return model

### 5.3: build a mixed model combining the neural network for tabular data and the CNN for images

In [20]:
def build_mixed_model(kernel_size = 2, pool_size = 2, tabular_shape = 19, tabular_activation_func = "relu", tabular_dense_list = [64], mixed_dense_list = [1024]):
    """
    Build a mixed model using Keras.

    Args:
    kernel_size: convolution layer kernel size
    pool_size: pooling layer pool size
    tabular_shape: dimension of the input
    tabular_activation_func: activation function to be used in the hidden dense layers
    tabular_dense_list: a list of dense layer units to be added to the tabular NN model
    mixed_dense_list: a list of dense layer units to be added to the mixed model

    Returns:
    model: A tf.keras model
    """
    
    #clear previous sessions
    tf.keras.backend.clear_session()

    #initialize two models
    cnn_model = build_cnn_model(kernel_size = kernel_size, pool_size = pool_size)
    tabular_model = build_tabular_model(input_dim = tabular_shape, activation_func = tabular_activation_func, tabular_dense_list = tabular_dense_list)

    #concatenate the two models by adding a concatenation layer
    combined_input = tf.keras.layers.concatenate([cnn_model.output, tabular_model.output])
    
    curr_layer = combined_input

    #add more dense layers based on dense_list
    for i in mixed_dense_list:
        curr_layer = tf.keras.layers.Dense(i, activation = 'relu')(curr_layer)
    
    #add output layer
    output = tf.keras.layers.Dense(1, activation = 'sigmoid')(curr_layer) 

    model = tf.keras.Model(inputs = [cnn_model.input, tabular_model.input], outputs = output)

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), 
                  loss = 'binary_crossentropy',  
                  metrics = ['accuracy'])
    
    return model

# Step 6: Fitting the Model Using Different Datasets

In [21]:
#create a list of dense layer unit lists for mixed model
mixed_dense_lists = [[1024], [1024, 512], [1024, 512, 256], [1024, 512, 256, 128], [1024, 512, 256, 128, 64], [1024, 512, 256, 128, 64, 32]]

#chose the tabular dense list that performed the best in previous testing
tabular_dense_lists = [[32, 64, 64]]

#chose the tabular activation function that peformed the 
activation_list = ["tanh"]

### 6.1: fit models using clean (original) dataset

In [22]:

clean_dict = {}

for mixed_dense_list in mixed_dense_lists:
    for tabular_dense_list in tabular_dense_lists:
        for activation_func in activation_list:
        
            print("---mixed dense list: " + str(mixed_dense_list) + ", tabular dense list: " + str(tabular_dense_list) + ", activation: " + activation_func + "---")
    
            model = build_mixed_model(kernel_size = 3,
                                      pool_size = 3,
                                      tabular_shape = train_tabular_clean.shape[1],
                                      tabular_activation_func = activation_func,
                                      tabular_dense_list = tabular_dense_list,
                                      mixed_dense_list = mixed_dense_list)
    
            history = model.fit(
                [x_train_clean,
                 train_tabular_clean],
                y_train_clean,
                validation_data = ([x_valid,
                                    valid_tabular_set],
                                   y_valid),
                epochs = 10,  
                batch_size = 128
            )
    
            clean_dict[(len(mixed_dense_list), len(tabular_dense_list), activation_func)] = [history.history['accuracy'][-1], history.history['val_accuracy'][-1]]


---mixed dense list: [1024], tabular dense list: [32, 64, 64], activation: tanh---
Epoch 1/10
47/47 [==============================] - 2s 30ms/step - loss: 7.5144 - accuracy: 0.7466 - val_loss: 0.3760 - val_accuracy: 0.8036
Epoch 2/10
47/47 [==============================] - 1s 30ms/step - loss: 0.3827 - accuracy: 0.8026 - val_loss: 0.3519 - val_accuracy: 0.8147
Epoch 3/10
47/47 [==============================] - 1s 29ms/step - loss: 0.3685 - accuracy: 0.8120 - val_loss: 0.3934 - val_accuracy: 0.8132
Epoch 4/10
47/47 [==============================] - 1s 29ms/step - loss: 0.3637 - accuracy: 0.8124 - val_loss: 0.3294 - val_accuracy: 0.8312
Epoch 5/10
47/47 [==============================] - 1s 27ms/step - loss: 0.3689 - accuracy: 0.8206 - val_loss: 0.3354 - val_accuracy: 0.8182
Epoch 6/10
47/47 [==============================] - 1s 27ms/step - loss: 0.3544 - accuracy: 0.8231 - val_loss: 0.3328 - val_accuracy: 0.8247
Epoch 7/10
47/47 [==============================] - 1s 28ms/step - loss

Epoch 1/10
47/47 [==============================] - 2s 31ms/step - loss: 4.1191 - accuracy: 0.7578 - val_loss: 0.3980 - val_accuracy: 0.8076
Epoch 2/10
47/47 [==============================] - 1s 30ms/step - loss: 0.3777 - accuracy: 0.8040 - val_loss: 0.3378 - val_accuracy: 0.8227
Epoch 3/10
47/47 [==============================] - 1s 29ms/step - loss: 0.3761 - accuracy: 0.8102 - val_loss: 0.3557 - val_accuracy: 0.8232
Epoch 4/10
47/47 [==============================] - 2s 33ms/step - loss: 0.3682 - accuracy: 0.8164 - val_loss: 0.3420 - val_accuracy: 0.8227
Epoch 5/10
47/47 [==============================] - 1s 31ms/step - loss: 0.3526 - accuracy: 0.8139 - val_loss: 0.3267 - val_accuracy: 0.8307
Epoch 6/10
47/47 [==============================] - 2s 33ms/step - loss: 0.3475 - accuracy: 0.8219 - val_loss: 0.3313 - val_accuracy: 0.8287
Epoch 7/10
47/47 [==============================] - 1s 31ms/step - loss: 0.3536 - accuracy: 0.8224 - val_loss: 0.3251 - val_accuracy: 0.8272
Epoch 8/10
47

Epoch 1/10
47/47 [==============================] - 2s 30ms/step - loss: 2.5298 - accuracy: 0.7794 - val_loss: 0.3684 - val_accuracy: 0.8152
Epoch 2/10
47/47 [==============================] - 1s 29ms/step - loss: 0.3668 - accuracy: 0.8080 - val_loss: 0.3381 - val_accuracy: 0.8257
Epoch 3/10
47/47 [==============================] - 1s 29ms/step - loss: 0.3607 - accuracy: 0.8135 - val_loss: 0.3453 - val_accuracy: 0.8122
Epoch 4/10
47/47 [==============================] - 1s 28ms/step - loss: 0.3650 - accuracy: 0.8107 - val_loss: 0.3330 - val_accuracy: 0.8237
Epoch 5/10
47/47 [==============================] - 1s 31ms/step - loss: 0.3509 - accuracy: 0.8211 - val_loss: 0.3385 - val_accuracy: 0.8252
Epoch 6/10
47/47 [==============================] - 1s 28ms/step - loss: 0.3483 - accuracy: 0.8278 - val_loss: 0.3318 - val_accuracy: 0.8247
Epoch 7/10
47/47 [==============================] - 1s 29ms/step - loss: 0.3419 - accuracy: 0.8278 - val_loss: 0.3220 - val_accuracy: 0.8398
Epoch 8/10
47

Epoch 1/10
47/47 [==============================] - 2s 30ms/step - loss: 1.1703 - accuracy: 0.7804 - val_loss: 0.3714 - val_accuracy: 0.8167
Epoch 2/10
47/47 [==============================] - 1s 28ms/step - loss: 0.3783 - accuracy: 0.8016 - val_loss: 0.3509 - val_accuracy: 0.8056
Epoch 3/10
47/47 [==============================] - 1s 29ms/step - loss: 0.3574 - accuracy: 0.8132 - val_loss: 0.3960 - val_accuracy: 0.7956
Epoch 4/10
47/47 [==============================] - 1s 30ms/step - loss: 0.3591 - accuracy: 0.8060 - val_loss: 0.3365 - val_accuracy: 0.8247
Epoch 5/10
47/47 [==============================] - 1s 29ms/step - loss: 0.3533 - accuracy: 0.8206 - val_loss: 0.3299 - val_accuracy: 0.8272
Epoch 6/10
47/47 [==============================] - 1s 29ms/step - loss: 0.3448 - accuracy: 0.8244 - val_loss: 0.3293 - val_accuracy: 0.8332
Epoch 7/10
47/47 [==============================] - 1s 31ms/step - loss: 0.3484 - accuracy: 0.8286 - val_loss: 0.3284 - val_accuracy: 0.8307
Epoch 8/10
47

Epoch 1/10
47/47 [==============================] - 2s 31ms/step - loss: 1.4605 - accuracy: 0.7651 - val_loss: 0.4008 - val_accuracy: 0.8056
Epoch 2/10
47/47 [==============================] - 1s 30ms/step - loss: 0.3780 - accuracy: 0.8068 - val_loss: 0.3500 - val_accuracy: 0.8177
Epoch 3/10
47/47 [==============================] - 1s 30ms/step - loss: 0.3614 - accuracy: 0.8139 - val_loss: 0.3489 - val_accuracy: 0.8177
Epoch 4/10
47/47 [==============================] - 1s 30ms/step - loss: 0.3725 - accuracy: 0.8102 - val_loss: 0.3688 - val_accuracy: 0.8302
Epoch 5/10
47/47 [==============================] - 1s 30ms/step - loss: 0.3526 - accuracy: 0.8217 - val_loss: 0.3285 - val_accuracy: 0.8312
Epoch 6/10
47/47 [==============================] - 1s 31ms/step - loss: 0.3442 - accuracy: 0.8229 - val_loss: 0.3317 - val_accuracy: 0.8312
Epoch 7/10
47/47 [==============================] - 1s 31ms/step - loss: 0.3485 - accuracy: 0.8237 - val_loss: 0.3438 - val_accuracy: 0.8302
Epoch 8/10
47

Epoch 1/10
47/47 [==============================] - 2s 30ms/step - loss: 1.0353 - accuracy: 0.7678 - val_loss: 0.3616 - val_accuracy: 0.8056
Epoch 2/10
47/47 [==============================] - 1s 30ms/step - loss: 0.3662 - accuracy: 0.8097 - val_loss: 0.3762 - val_accuracy: 0.8026
Epoch 3/10
47/47 [==============================] - 1s 29ms/step - loss: 0.3573 - accuracy: 0.8149 - val_loss: 0.3549 - val_accuracy: 0.8247
Epoch 4/10
47/47 [==============================] - 1s 30ms/step - loss: 0.3593 - accuracy: 0.8214 - val_loss: 0.3319 - val_accuracy: 0.8302
Epoch 5/10
47/47 [==============================] - 1s 28ms/step - loss: 0.3460 - accuracy: 0.8252 - val_loss: 0.3393 - val_accuracy: 0.8242
Epoch 6/10
47/47 [==============================] - 1s 30ms/step - loss: 0.3422 - accuracy: 0.8273 - val_loss: 0.3434 - val_accuracy: 0.8227
Epoch 7/10
47/47 [==============================] - 1s 29ms/step - loss: 0.3379 - accuracy: 0.8323 - val_loss: 0.3389 - val_accuracy: 0.8112
Epoch 8/10
47

### 6.2: fit models using oversample dataset

In [23]:

oversample_dict = {}

for mixed_dense_list in mixed_dense_lists:
    for tabular_dense_list in tabular_dense_lists:
        for activation_func in activation_list:
        
            print("---mixed dense list: " + str(mixed_dense_list) + ", tabular dense list: " + str(tabular_dense_list) + ", activation: " + activation_func + "---")
    
            model = build_mixed_model(kernel_size = 3,
                                      pool_size = 3,
                                      tabular_shape = train_tabular_oversample.shape[1],
                                      tabular_activation_func = activation_func,
                                      tabular_dense_list = tabular_dense_list,
                                      mixed_dense_list = mixed_dense_list)
    
            history = model.fit(
                [x_train_oversample,
                 train_tabular_oversample],
                y_train_oversample,
                validation_data = ([x_valid,
                                    valid_tabular_set],
                                   y_valid),
                epochs=10,  
                batch_size=128
            )

            oversample_dict[(len(mixed_dense_list), len(tabular_dense_list), activation_func)] = [history.history['accuracy'][-1], history.history['val_accuracy'][-1]]


---mixed dense list: [1024], tabular dense list: [32, 64, 64], activation: tanh---


Epoch 1/10
76/76 [==============================] - 2s 25ms/step - loss: 7.1060 - accuracy: 0.6594 - val_loss: 0.4911 - val_accuracy: 0.7499
Epoch 2/10
76/76 [==============================] - 2s 24ms/step - loss: 0.4894 - accuracy: 0.7631 - val_loss: 0.3988 - val_accuracy: 0.7810
Epoch 3/10
76/76 [==============================] - 2s 25ms/step - loss: 0.4668 - accuracy: 0.7764 - val_loss: 0.3810 - val_accuracy: 0.7760
Epoch 4/10
76/76 [==============================] - 2s 25ms/step - loss: 0.4551 - accuracy: 0.7828 - val_loss: 0.3870 - val_accuracy: 0.7735
Epoch 5/10
76/76 [==============================] - 2s 25ms/step - loss: 0.4422 - accuracy: 0.7832 - val_loss: 0.4767 - val_accuracy: 0.7268
Epoch 6/10
76/76 [==============================] - 2s 24ms/step - loss: 0.4295 - accuracy: 0.7922 - val_loss: 0.4204 - val_accuracy: 0.7534
Epoch 7/10
76/76 [==============================] - 2s 25ms/step - loss: 0.4281 - accuracy: 0.7949 - val_loss: 0.3883 - val_accuracy: 0.7700
Epoch 8/10
76

---mixed dense list: [1024, 512], tabular dense list: [32, 64, 64], activation: tanh---
Epoch 1/10
76/76 [==============================] - 2s 26ms/step - loss: 4.2251 - accuracy: 0.6503 - val_loss: 0.4460 - val_accuracy: 0.7534
Epoch 2/10
76/76 [==============================] - 2s 26ms/step - loss: 0.4703 - accuracy: 0.7727 - val_loss: 0.3833 - val_accuracy: 0.7740
Epoch 3/10
76/76 [==============================] - 2s 25ms/step - loss: 0.4502 - accuracy: 0.7845 - val_loss: 0.4256 - val_accuracy: 0.7398
Epoch 4/10
76/76 [==============================] - 2s 27ms/step - loss: 0.4330 - accuracy: 0.7921 - val_loss: 0.4067 - val_accuracy: 0.7685
Epoch 5/10
76/76 [==============================] - 2s 28ms/step - loss: 0.4267 - accuracy: 0.7988 - val_loss: 0.4159 - val_accuracy: 0.7514
Epoch 6/10
76/76 [==============================] - 2s 27ms/step - loss: 0.4188 - accuracy: 0.8022 - val_loss: 0.3809 - val_accuracy: 0.7850
Epoch 7/10
76/76 [==============================] - 2s 26ms/step -

Epoch 1/10
76/76 [==============================] - 2s 29ms/step - loss: 2.3963 - accuracy: 0.6650 - val_loss: 0.4270 - val_accuracy: 0.7564
Epoch 2/10
76/76 [==============================] - 2s 27ms/step - loss: 0.4716 - accuracy: 0.7727 - val_loss: 0.5469 - val_accuracy: 0.6936
Epoch 3/10
76/76 [==============================] - 2s 28ms/step - loss: 0.4603 - accuracy: 0.7778 - val_loss: 0.4606 - val_accuracy: 0.7308
Epoch 4/10
76/76 [==============================] - 2s 28ms/step - loss: 0.4366 - accuracy: 0.7913 - val_loss: 0.4375 - val_accuracy: 0.7353
Epoch 5/10
76/76 [==============================] - 2s 28ms/step - loss: 0.4392 - accuracy: 0.7897 - val_loss: 0.4030 - val_accuracy: 0.7519
Epoch 6/10
76/76 [==============================] - 2s 32ms/step - loss: 0.4154 - accuracy: 0.8016 - val_loss: 0.3702 - val_accuracy: 0.7896
Epoch 7/10
76/76 [==============================] - 3s 38ms/step - loss: 0.4029 - accuracy: 0.8073 - val_loss: 0.3343 - val_accuracy: 0.8192
Epoch 8/10
76

Epoch 1/10
76/76 [==============================] - 3s 31ms/step - loss: 1.2096 - accuracy: 0.6800 - val_loss: 0.4150 - val_accuracy: 0.7760
Epoch 2/10
76/76 [==============================] - 2s 31ms/step - loss: 0.4671 - accuracy: 0.7760 - val_loss: 0.4832 - val_accuracy: 0.7238
Epoch 3/10
76/76 [==============================] - 3s 34ms/step - loss: 0.4558 - accuracy: 0.7839 - val_loss: 0.6117 - val_accuracy: 0.6514
Epoch 4/10
76/76 [==============================] - 2s 31ms/step - loss: 0.4438 - accuracy: 0.7867 - val_loss: 0.4065 - val_accuracy: 0.7519
Epoch 5/10
76/76 [==============================] - 2s 29ms/step - loss: 0.4382 - accuracy: 0.7913 - val_loss: 0.4641 - val_accuracy: 0.7273
Epoch 6/10
76/76 [==============================] - 2s 30ms/step - loss: 0.4188 - accuracy: 0.7978 - val_loss: 0.4360 - val_accuracy: 0.7534
Epoch 7/10
76/76 [==============================] - 2s 30ms/step - loss: 0.4152 - accuracy: 0.8005 - val_loss: 0.4115 - val_accuracy: 0.7378
Epoch 8/10
76

Epoch 1/10
76/76 [==============================] - 2s 29ms/step - loss: 1.1768 - accuracy: 0.6777 - val_loss: 0.4036 - val_accuracy: 0.7670
Epoch 2/10
76/76 [==============================] - 2s 30ms/step - loss: 0.4729 - accuracy: 0.7712 - val_loss: 0.3887 - val_accuracy: 0.7750
Epoch 3/10
76/76 [==============================] - 3s 35ms/step - loss: 0.4600 - accuracy: 0.7765 - val_loss: 0.4042 - val_accuracy: 0.7569
Epoch 4/10
76/76 [==============================] - 2s 31ms/step - loss: 0.4527 - accuracy: 0.7811 - val_loss: 0.4209 - val_accuracy: 0.7574
Epoch 5/10
76/76 [==============================] - 2s 29ms/step - loss: 0.4427 - accuracy: 0.7893 - val_loss: 0.4641 - val_accuracy: 0.7338
Epoch 6/10
76/76 [==============================] - 2s 30ms/step - loss: 0.4264 - accuracy: 0.7959 - val_loss: 0.5114 - val_accuracy: 0.7258
Epoch 7/10
76/76 [==============================] - 2s 28ms/step - loss: 0.4338 - accuracy: 0.7907 - val_loss: 0.3831 - val_accuracy: 0.7936
Epoch 8/10
76

---mixed dense list: [1024, 512, 256, 128, 64, 32], tabular dense list: [32, 64, 64], activation: tanh---
Epoch 1/10
76/76 [==============================] - 2s 29ms/step - loss: 0.8470 - accuracy: 0.6549 - val_loss: 0.4163 - val_accuracy: 0.7649
Epoch 2/10
76/76 [==============================] - 2s 28ms/step - loss: 0.4832 - accuracy: 0.7700 - val_loss: 0.4916 - val_accuracy: 0.7037
Epoch 3/10
76/76 [==============================] - 2s 27ms/step - loss: 0.4546 - accuracy: 0.7780 - val_loss: 0.3975 - val_accuracy: 0.7524
Epoch 4/10
76/76 [==============================] - 2s 27ms/step - loss: 0.4363 - accuracy: 0.7914 - val_loss: 0.3811 - val_accuracy: 0.7639
Epoch 5/10
76/76 [==============================] - 2s 28ms/step - loss: 0.4291 - accuracy: 0.7930 - val_loss: 0.3865 - val_accuracy: 0.7820
Epoch 6/10
76/76 [==============================] - 2s 28ms/step - loss: 0.4122 - accuracy: 0.8039 - val_loss: 0.4598 - val_accuracy: 0.7338
Epoch 7/10
76/76 [==============================

### 6.3: fit models using image augmentation dataset

In [24]:

image_augm_dict = {}

for mixed_dense_list in mixed_dense_lists:
    for tabular_dense_list in tabular_dense_lists:
        for activation_func in activation_list:
        
            print("---mixed dense list: " + str(mixed_dense_list) + ", tabular dense list: " + str(tabular_dense_list) + ", activation: " + activation_func + "---")
    
            x_train_augm, tabular_augm, y_train_augm = aug_image(x_train_clean,
                                                                 train_tabular_clean,
                                                                 y_train_clean,
                                                                 contrast_factor = 1,
                                                                 delta = 0.1,
                                                                 flip_flag = True)
    
            model = build_mixed_model(kernel_size = 3,
                                      pool_size = 3,
                                      tabular_shape = tabular_augm.shape[1],
                                      tabular_activation_func = activation_func,
                                      tabular_dense_list = tabular_dense_list,
                                      mixed_dense_list = mixed_dense_list)
    
            history = model.fit(
                [x_train_augm,
                 tabular_augm],
                y_train_augm,
                validation_data = ([x_valid_augm,
                                    valid_tabular_set],
                                   y_valid),
                epochs=10,  
                batch_size=128
            )

            image_augm_dict[(len(mixed_dense_list), len(tabular_dense_list), activation_func)] = [history.history['accuracy'][-1], history.history['val_accuracy'][-1]]
        

---mixed dense list: [1024], tabular dense list: [32, 64, 64], activation: tanh---


Epoch 1/10
94/94 [==============================] - 3s 25ms/step - loss: 1.2147 - accuracy: 0.7903 - val_loss: 0.3944 - val_accuracy: 0.8071
Epoch 2/10
94/94 [==============================] - 2s 24ms/step - loss: 0.3927 - accuracy: 0.8057 - val_loss: 0.3856 - val_accuracy: 0.8056
Epoch 3/10
94/94 [==============================] - 2s 24ms/step - loss: 0.3888 - accuracy: 0.8076 - val_loss: 0.3525 - val_accuracy: 0.8187
Epoch 4/10
94/94 [==============================] - 2s 25ms/step - loss: 0.3702 - accuracy: 0.8100 - val_loss: 0.3613 - val_accuracy: 0.8142
Epoch 5/10
94/94 [==============================] - 2s 25ms/step - loss: 0.3652 - accuracy: 0.8104 - val_loss: 0.3636 - val_accuracy: 0.8127
Epoch 6/10
94/94 [==============================] - 2s 24ms/step - loss: 0.3583 - accuracy: 0.8140 - val_loss: 0.3621 - val_accuracy: 0.8197
Epoch 7/10
94/94 [==============================] - 2s 24ms/step - loss: 0.3587 - accuracy: 0.8165 - val_loss: 0.3434 - val_accuracy: 0.8152
Epoch 8/10
94

Epoch 1/10
94/94 [==============================] - 3s 27ms/step - loss: 0.9511 - accuracy: 0.7944 - val_loss: 0.3925 - val_accuracy: 0.8122
Epoch 2/10
94/94 [==============================] - 3s 28ms/step - loss: 0.3887 - accuracy: 0.8022 - val_loss: 0.4038 - val_accuracy: 0.8142
Epoch 3/10
94/94 [==============================] - 3s 29ms/step - loss: 0.3951 - accuracy: 0.8029 - val_loss: 0.3576 - val_accuracy: 0.8056
Epoch 4/10
94/94 [==============================] - 3s 30ms/step - loss: 0.3731 - accuracy: 0.8076 - val_loss: 0.3523 - val_accuracy: 0.8137
Epoch 5/10
94/94 [==============================] - 3s 32ms/step - loss: 0.3772 - accuracy: 0.8065 - val_loss: 0.3407 - val_accuracy: 0.8086
Epoch 6/10
94/94 [==============================] - 3s 27ms/step - loss: 0.3693 - accuracy: 0.8094 - val_loss: 0.3421 - val_accuracy: 0.8086
Epoch 7/10
94/94 [==============================] - 3s 28ms/step - loss: 0.3606 - accuracy: 0.8135 - val_loss: 0.3358 - val_accuracy: 0.8066
Epoch 8/10
94

Epoch 1/10
94/94 [==============================] - 3s 31ms/step - loss: 1.2059 - accuracy: 0.7891 - val_loss: 0.3963 - val_accuracy: 0.8076
Epoch 2/10
94/94 [==============================] - 3s 28ms/step - loss: 0.3962 - accuracy: 0.8058 - val_loss: 0.3945 - val_accuracy: 0.8056
Epoch 3/10
94/94 [==============================] - 3s 30ms/step - loss: 0.3905 - accuracy: 0.8031 - val_loss: 0.3731 - val_accuracy: 0.8041
Epoch 4/10
94/94 [==============================] - 3s 27ms/step - loss: 0.3869 - accuracy: 0.8052 - val_loss: 0.3590 - val_accuracy: 0.8202
Epoch 5/10
94/94 [==============================] - 3s 29ms/step - loss: 0.3690 - accuracy: 0.8082 - val_loss: 0.3413 - val_accuracy: 0.8061
Epoch 6/10
94/94 [==============================] - 3s 31ms/step - loss: 0.3845 - accuracy: 0.8063 - val_loss: 0.3993 - val_accuracy: 0.8086
Epoch 7/10
94/94 [==============================] - 3s 31ms/step - loss: 0.3992 - accuracy: 0.8066 - val_loss: 0.3772 - val_accuracy: 0.8056
Epoch 8/10
94

Epoch 1/10
94/94 [==============================] - 3s 34ms/step - loss: 0.7991 - accuracy: 0.7947 - val_loss: 0.3982 - val_accuracy: 0.8056
Epoch 2/10
94/94 [==============================] - 3s 30ms/step - loss: 0.3875 - accuracy: 0.8062 - val_loss: 0.3665 - val_accuracy: 0.8056
Epoch 3/10
94/94 [==============================] - 3s 31ms/step - loss: 0.3962 - accuracy: 0.8062 - val_loss: 0.3645 - val_accuracy: 0.8056
Epoch 4/10
94/94 [==============================] - 3s 31ms/step - loss: 0.3700 - accuracy: 0.8101 - val_loss: 0.3546 - val_accuracy: 0.8096
Epoch 5/10
94/94 [==============================] - 3s 29ms/step - loss: 0.3692 - accuracy: 0.8108 - val_loss: 0.3629 - val_accuracy: 0.8051
Epoch 6/10
94/94 [==============================] - 3s 30ms/step - loss: 0.3641 - accuracy: 0.8091 - val_loss: 0.3461 - val_accuracy: 0.8101
Epoch 7/10
94/94 [==============================] - 3s 29ms/step - loss: 0.3569 - accuracy: 0.8129 - val_loss: 0.3309 - val_accuracy: 0.8091
Epoch 8/10
94

Epoch 1/10
94/94 [==============================] - 3s 29ms/step - loss: 0.7088 - accuracy: 0.7916 - val_loss: 0.3988 - val_accuracy: 0.8056
Epoch 2/10
94/94 [==============================] - 3s 31ms/step - loss: 0.4040 - accuracy: 0.8056 - val_loss: 0.4136 - val_accuracy: 0.8056
Epoch 3/10
94/94 [==============================] - 3s 30ms/step - loss: 0.3973 - accuracy: 0.8066 - val_loss: 0.3562 - val_accuracy: 0.8056
Epoch 4/10
94/94 [==============================] - 3s 30ms/step - loss: 0.3833 - accuracy: 0.8036 - val_loss: 0.3866 - val_accuracy: 0.8056
Epoch 5/10
94/94 [==============================] - 3s 30ms/step - loss: 0.3757 - accuracy: 0.8052 - val_loss: 0.3483 - val_accuracy: 0.8076
Epoch 6/10
94/94 [==============================] - 3s 31ms/step - loss: 0.3750 - accuracy: 0.8049 - val_loss: 0.3477 - val_accuracy: 0.8056
Epoch 7/10
94/94 [==============================] - 3s 30ms/step - loss: 0.3712 - accuracy: 0.8079 - val_loss: 0.3531 - val_accuracy: 0.8056
Epoch 8/10
94

Epoch 1/10
94/94 [==============================] - 3s 32ms/step - loss: 0.6670 - accuracy: 0.7895 - val_loss: 0.4009 - val_accuracy: 0.8056
Epoch 2/10
94/94 [==============================] - 3s 30ms/step - loss: 0.4050 - accuracy: 0.8065 - val_loss: 0.3966 - val_accuracy: 0.8056
Epoch 3/10
94/94 [==============================] - 3s 29ms/step - loss: 0.3940 - accuracy: 0.8043 - val_loss: 0.3711 - val_accuracy: 0.8056
Epoch 4/10
94/94 [==============================] - 3s 30ms/step - loss: 0.3817 - accuracy: 0.8057 - val_loss: 0.3925 - val_accuracy: 0.8051
Epoch 5/10
94/94 [==============================] - 3s 29ms/step - loss: 0.3839 - accuracy: 0.8063 - val_loss: 0.3552 - val_accuracy: 0.8081
Epoch 6/10
94/94 [==============================] - 3s 31ms/step - loss: 0.3702 - accuracy: 0.8055 - val_loss: 0.3644 - val_accuracy: 0.8056
Epoch 7/10
94/94 [==============================] - 3s 30ms/step - loss: 0.3746 - accuracy: 0.8080 - val_loss: 0.3535 - val_accuracy: 0.8056
Epoch 8/10
94

### 6.4: fit models using oversample + image augmentation dataset

In [25]:

image_augm_over_dict = {}

for mixed_dense_list in mixed_dense_lists:
    for tabular_dense_list in tabular_dense_lists:
        for activation_func in activation_list:
        
            print("---mixed dense list: " + str(mixed_dense_list) + ", tabular dense list: " + str(tabular_dense_list) + ", activation: " + activation_func + "---")

            x_train_oversample_augm, tabular_oversample_augm, y_train_oversample_augm = aug_image(x_train_oversample,
                                                                                                  train_tabular_oversample,
                                                                                                  y_train_oversample,
                                                                                                  contrast_factor = 1,
                                                                                                  delta = 0.1,
                                                                                                  flip_flag = True)
    
            model = build_mixed_model(kernel_size = 3,
                                      pool_size = 3,
                                      tabular_shape = tabular_oversample_augm.shape[1],
                                      tabular_activation_func = activation_func,
                                      tabular_dense_list = tabular_dense_list,
                                      mixed_dense_list = mixed_dense_list)
    
            history = model.fit(
                [x_train_oversample_augm,
                 tabular_oversample_augm],
                y_train_oversample_augm,
                validation_data = ([x_valid_augm,
                                    valid_tabular_set],
                                   y_valid),
                epochs=10,  
                batch_size=128
            )

            image_augm_over_dict[(len(mixed_dense_list), len(tabular_dense_list), activation_func)] = [history.history['accuracy'][-1], history.history['val_accuracy'][-1]]
        

---mixed dense list: [1024], tabular dense list: [32, 64, 64], activation: tanh---


Epoch 1/10
151/151 [==============================] - 4s 25ms/step - loss: 1.7164 - accuracy: 0.7169 - val_loss: 0.5003 - val_accuracy: 0.7484
Epoch 2/10
151/151 [==============================] - 4s 26ms/step - loss: 0.4877 - accuracy: 0.7673 - val_loss: 0.4705 - val_accuracy: 0.7504
Epoch 3/10
151/151 [==============================] - 4s 26ms/step - loss: 0.4571 - accuracy: 0.7816 - val_loss: 0.4412 - val_accuracy: 0.7443
Epoch 4/10
151/151 [==============================] - 4s 25ms/step - loss: 0.4345 - accuracy: 0.7944 - val_loss: 0.4278 - val_accuracy: 0.7519
Epoch 5/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4322 - accuracy: 0.7946 - val_loss: 0.4579 - val_accuracy: 0.7323
Epoch 6/10
151/151 [==============================] - 4s 26ms/step - loss: 0.4227 - accuracy: 0.7980 - val_loss: 0.4240 - val_accuracy: 0.7594
Epoch 7/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4106 - accuracy: 0.8046 - val_loss: 0.4445 - val_accuracy: 0.7524

Epoch 1/10
151/151 [==============================] - 4s 27ms/step - loss: 1.1189 - accuracy: 0.7147 - val_loss: 0.5062 - val_accuracy: 0.7454
Epoch 2/10
151/151 [==============================] - 4s 28ms/step - loss: 0.4788 - accuracy: 0.7668 - val_loss: 0.4485 - val_accuracy: 0.7243
Epoch 3/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4491 - accuracy: 0.7844 - val_loss: 0.4288 - val_accuracy: 0.7484
Epoch 4/10
151/151 [==============================] - 4s 29ms/step - loss: 0.4371 - accuracy: 0.7904 - val_loss: 0.5565 - val_accuracy: 0.6705
Epoch 5/10
151/151 [==============================] - 4s 26ms/step - loss: 0.4199 - accuracy: 0.7995 - val_loss: 0.4490 - val_accuracy: 0.7323
Epoch 6/10
151/151 [==============================] - 4s 26ms/step - loss: 0.4035 - accuracy: 0.8069 - val_loss: 0.4282 - val_accuracy: 0.7514
Epoch 7/10
151/151 [==============================] - 4s 27ms/step - loss: 0.3874 - accuracy: 0.8157 - val_loss: 0.4450 - val_accuracy: 0.7529

Epoch 1/10
151/151 [==============================] - 4s 27ms/step - loss: 0.9529 - accuracy: 0.7085 - val_loss: 0.4872 - val_accuracy: 0.7559
Epoch 2/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4870 - accuracy: 0.7652 - val_loss: 0.4503 - val_accuracy: 0.7293
Epoch 3/10
151/151 [==============================] - 4s 26ms/step - loss: 0.4601 - accuracy: 0.7795 - val_loss: 0.4563 - val_accuracy: 0.7273
Epoch 4/10
151/151 [==============================] - 4s 26ms/step - loss: 0.4469 - accuracy: 0.7839 - val_loss: 0.4795 - val_accuracy: 0.7152
Epoch 5/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4399 - accuracy: 0.7899 - val_loss: 0.4824 - val_accuracy: 0.7167
Epoch 6/10
151/151 [==============================] - 4s 28ms/step - loss: 0.4234 - accuracy: 0.7992 - val_loss: 0.4543 - val_accuracy: 0.7358
Epoch 7/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4243 - accuracy: 0.7973 - val_loss: 0.4485 - val_accuracy: 0.7459

Epoch 1/10
151/151 [==============================] - 4s 26ms/step - loss: 0.7723 - accuracy: 0.7032 - val_loss: 0.4958 - val_accuracy: 0.7388
Epoch 2/10
151/151 [==============================] - 4s 26ms/step - loss: 0.5114 - accuracy: 0.7542 - val_loss: 0.4481 - val_accuracy: 0.7544
Epoch 3/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4771 - accuracy: 0.7703 - val_loss: 0.4709 - val_accuracy: 0.7353
Epoch 4/10
151/151 [==============================] - 4s 26ms/step - loss: 0.4569 - accuracy: 0.7790 - val_loss: 0.5053 - val_accuracy: 0.6781
Epoch 5/10
151/151 [==============================] - 4s 26ms/step - loss: 0.5368 - accuracy: 0.7362 - val_loss: 0.6485 - val_accuracy: 0.5726
Epoch 6/10
151/151 [==============================] - 4s 26ms/step - loss: 0.4657 - accuracy: 0.7735 - val_loss: 0.4686 - val_accuracy: 0.7192
Epoch 7/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4447 - accuracy: 0.7873 - val_loss: 0.4569 - val_accuracy: 0.7228

Epoch 1/10
151/151 [==============================] - 4s 28ms/step - loss: 0.8476 - accuracy: 0.7040 - val_loss: 0.5142 - val_accuracy: 0.7353
Epoch 2/10
151/151 [==============================] - 4s 27ms/step - loss: 0.5241 - accuracy: 0.7454 - val_loss: 0.4284 - val_accuracy: 0.7685
Epoch 3/10
151/151 [==============================] - 4s 26ms/step - loss: 0.4892 - accuracy: 0.7633 - val_loss: 0.4781 - val_accuracy: 0.7298
Epoch 4/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4651 - accuracy: 0.7749 - val_loss: 0.4532 - val_accuracy: 0.7152
Epoch 5/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4481 - accuracy: 0.7855 - val_loss: 0.5530 - val_accuracy: 0.6735
Epoch 6/10
151/151 [==============================] - 4s 26ms/step - loss: 0.4337 - accuracy: 0.7924 - val_loss: 0.4183 - val_accuracy: 0.7459
Epoch 7/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4206 - accuracy: 0.8007 - val_loss: 0.4201 - val_accuracy: 0.7730

Epoch 1/10
151/151 [==============================] - 4s 27ms/step - loss: 0.7203 - accuracy: 0.6941 - val_loss: 0.5177 - val_accuracy: 0.7318
Epoch 2/10
151/151 [==============================] - 4s 27ms/step - loss: 0.5121 - accuracy: 0.7483 - val_loss: 0.4218 - val_accuracy: 0.7649
Epoch 3/10
151/151 [==============================] - 4s 28ms/step - loss: 0.4768 - accuracy: 0.7718 - val_loss: 0.4712 - val_accuracy: 0.7072
Epoch 4/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4538 - accuracy: 0.7841 - val_loss: 0.5600 - val_accuracy: 0.6409
Epoch 5/10
151/151 [==============================] - 4s 27ms/step - loss: 0.4483 - accuracy: 0.7817 - val_loss: 0.4767 - val_accuracy: 0.7117
Epoch 6/10
151/151 [==============================] - 4s 28ms/step - loss: 0.4372 - accuracy: 0.7897 - val_loss: 0.4189 - val_accuracy: 0.7408
Epoch 7/10
151/151 [==============================] - 4s 28ms/step - loss: 0.4312 - accuracy: 0.7950 - val_loss: 0.4754 - val_accuracy: 0.7107

# Step 7: Output Results

In [26]:
import csv

clean_table = []

for key in clean_dict:
    clean_table.append([key[0], key[1], key[2], clean_dict[key][0], clean_dict[key][1]])

with open('clean_mixed_binary.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(clean_table)
    

In [27]:

oversample_table = []

for key in oversample_dict:
    oversample_table.append([key[0], key[1], key[2], oversample_dict[key][0], oversample_dict[key][1]])

with open('oversample_mixed_binary.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(oversample_table)
    

In [28]:

image_augm_table = []

for key in image_augm_dict:
    image_augm_table.append([key[0], key[1], key[2], image_augm_dict[key][0], image_augm_dict[key][1]])

with open('image_augm_mixed_binary.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(image_augm_table)
    

In [29]:

image_augm_over_table = []

for key in image_augm_over_dict:
    image_augm_over_table.append([key[0], key[1], key[2], image_augm_over_dict[key][0], image_augm_over_dict[key][1]])

with open('image_augm_over_mixed_binary.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(image_augm_over_table)
    